In [7]:
%%writefile murderpeople.lp
{ peopleCount(4..10)} = 1.
idNumber(I) :- I=1..K, peopleCount(K).

{identity(I, N) : personID(N)} = 1 :- idNumber(I).
:- identity(I1,N), identity(I2,N), I1 != I2, personID(N).
person(N) :- identity(I,N).

%Setup
0 < {has_relationship(I1,I2,R) : relationship(R,_,_), person(I2), I1 != I2} < (K-1) :- person(I1), peopleCount(K).
has_relationship(I1,I2,R) :- relationship(R,undirected,_), has_relationship(I2,I1,R).

{has_opinion(I1,I2,O) : opinion(O)} = 1 :- person(I1), person(I2), I1 != I2.
    
{has_profession(I, P) : profession(P,_)} = 1 :- person(I).
{has_profession(I, P) : person(I)} <= N :- profession(P,N).
{has_modifier(I, P) : modifier(P,_)} <= 1 :- person(I).
{has_modifier(I, P) : person(I)} <= N :- modifier(P,N).

modifier_count(N) :- N = #count{person(I): has_modifier(I,_)}.
:- modifier_count(N), peopleCount(K), N < K/2. %Half or more of the characters have modifiers.
    
{personality(N,"primary",T) : primarytrope(T)}=1 :- identity(I, N).
{personality(N,"secondary",T) : trope(T)}=1 :- identity(I, N).
    
{has_value(N,1,V) : value(V)} = 1 :- person(N).
{has_value(N,2,V) : value(V)} = 1 :- person(N).
    
:- has_value(N,1,V), has_value(N,2,V), person(N), value(V).

%Validation

:- has_relationship(I1,I2,"professor_of"), person(I1), person(I2), not has_profession(I1,"professor").
:- has_relationship(I1,I2,"professor_of"), person(I1), person(I2), not has_profession(I2,"student").
:- has_relationship(I1,I2,"working_for"), has_relationship(I2,I1,"working_for").
:- has_relationship(I1,I2,"mentor_of"), has_relationship(I2,I1,"mentor_of").
    
relationshipCount(R,N) :- relationship(R, _,_), N = #count{peoplePair((I1,I2)): has_relationship(I1,I2,R)}.

personal_relationshipCount(P,N,1) :- person(P), N = #count{ r_ship_names(R): has_relationship(P,P2,R), person(P2)}.
personal_relationshipCount(P,N,2) :- person(P), N = #count{ r_ship_names(R): has_relationship(P2,P,R), person(P2)}.
personal_relationshipCount(P,N) :- N = N1 + N2, personal_relationshipCount(P,N1,1), personal_relationshipCount(P,N2,2).
    
:- relationshipCount(R,N), peopleCount(K), N >= K/2.
:- personal_relationshipCount(P,N), peopleCount(K), N > K.
:- person(P1), person(P2), has_relationship(P1,P2,R1), has_relationship(P1,P2,R2), R1 != R2.
:- has_relationship(I1,I1,T), person(I1).

:- personality(P,"primary",T), personality(P,"secondary",T).
:- personality(P1,"primary",T), personality(P2,"primary",T), P1 != P2.
    
%Tuning rules
{has_relationship(I1,I2,"secret_crush") : person(I1), person(I2)} < 2.

%Now, with these values and the total number of people, we can control how well people get along on average!
peoplePair((I1,I2)) :- person(I1), person(I2), I1 != I2.
likes(N) :- N = #count{peoplePair((I1,I2)) : has_opinion(I1,I2,"likes")}.
dislikes(N) :- N = #count{peoplePair((I1,I2)) : has_opinion(I1,I2,"dislikes")}.
neutral(N) :- N = #count{peoplePair((I1,I2)) : has_opinion(I1,I2,"neutral")}.  


Overwriting murderpeople.lp


In [3]:
%%writefile identities.lp
personID("Alice").
personID("Bob").
personID("Cindy").
personID("Dan").
personID("Eliza").
personID("Fred").
personID("Georgia").
personID("Henry").
personID("Isabel").
personID("John").
personID("Kate").
personID("Louis").

relationship("coauthors", undirected,professional).
relationship("working_for", directed,professional).
relationship("mutual_rivals", undirected,professional).
relationship("rivals", directed,professional).
relationship("mentor_of", directed,professional).
relationship("professor_of", directed,professional).

relationship("secret_crush", directed,romantic).
relationship("dating", undirected,romantic).
relationship("exes", undirected,romantic).

relationship("sibling_of", undirected,family).
relationship("married", undirected,family).

r_ship_names(N) :- relationship(N,_,_).

opinion("likes").
opinion("neutral").
opinion("dislikes").

profession("professor",2).
profession("student",4).
profession("observatory_caretaker",1).
profession("groundskeeper",1).
profession("psychic",1).
profession("reporter",1).
profession("skier",2).
profession("astronomer",2).
profession("volunteer",1).
profession("high schooler",1).
profession("giftshopkeep",1).
profession("security",1).
profession("tourist",1).
profession("tour guide",1).
profession("archivist",1).

modifier("is_rich",1).
modifier("in_debt",1).
modifier("is_dying",1).
modifier("is_important_online",1).
modifier("celebritiy_status",1).
modifier("outsider",1).
modifier("secret_expert",1).
modifier("in_hiding",1).
modifier("has_funding",1).
modifier("has_theory_about_astronomical_event",2).
modifier("is_secretly_right_about_the_science",1).
modifier("protective_of_observatory",2).
modifier("hates_this_place",1).

trope(X) :- primarytrope(X).

primarytrope("Absent-Minded Professor").
primarytrope("Big Fun").
primarytrope("Boss").
primarytrope("Bad-to-the-bone").
primarytrope("Parent Figure").
primarytrope("Innocent").
primarytrope("Clown").
primarytrope("Nerd").
primarytrope("Tortured Artist").
primarytrope("Gentle Giant").
primarytrope("Scruffy and Gruff").
primarytrope("Regal Presence").
primarytrope("Seductive").
primarytrope("Dumb Muscle").
primarytrope("Elderly Master").
primarytrope("Know-it-all").
primarytrope("Strict/By-the-books").
primarytrope("Rugged").
primarytrope("Lone Wolf").
primarytrope("Athlete").
primarytrope("Eccentric").

trope("Peacemaker").
trope("Pessimist").
trope("Optimist").
trope("Introvert").
trope("Extrovert").
trope("Jerk").
trope("Resigned").
trope("Loyalist").
trope("Friend Next Door").
trope("Child").
trope("Outspoken").
trope("Mousey/Shy").
trope("Conscience").
trope("Imposter/Pretender").
trope("Side Kick").
trope("Astrology Buff").
trope("Socially Awkward/Misses Cues").

value("Science").
value("Survival").
value("Communalism").
value("Funding").
value("Comfort").
value("Order").
value("Faith").
value("Progress").

Overwriting identities.lp


In [4]:
%%writefile observatory.lp

location(kitchen).
location(researchLab).
%This file left as an exercise for someone not me.

Overwriting observatory.lp


In [5]:
%%writefile show.lp
#show peopleCount/1.
#show has_relationship/3.
#show has_profession/2.
#show has_modifier/2.
#show personality/3.
#show has_value/3.
#show modifier_count/1.
#show personal_relationshipCount/2.
peopleCount(10).

Overwriting show.lp


In [8]:
import json
lines = !clingo murderpeople.lp identities.lp show.lp --outf=2
try:
    json_text = json.loads('\n'.join(lines))
except:
    #print(lines)
    i = lines.index('{')
    json_text = json.loads('\n'.join(lines[i:]))
try:
    totalWitnesses = len(json_text['Call'][0]['Witnesses'])
    print("Total: " + str(totalWitnesses))
    print()
    for witness in json_text['Call'][0]['Witnesses']:
        print( "New Instance")
        for line in sorted(witness['Value']):
            print(line + ".")
        print("")
except:
    print("No Witnesses")
    print(lines)

Total: 1

New Instance
has_modifier("Alice","outsider").
has_modifier("Bob","celebritiy_status").
has_modifier("Georgia","has_funding").
has_modifier("Henry","secret_expert").
has_modifier("Louis","in_debt").
has_profession("Alice","giftshopkeep").
has_profession("Bob","student").
has_profession("Cindy","skier").
has_profession("Fred","professor").
has_profession("Georgia","student").
has_profession("Henry","professor").
has_profession("Isabel","student").
has_profession("John","skier").
has_profession("Kate","astronomer").
has_profession("Louis","student").
has_relationship("Alice","Cindy","mutual_rivals").
has_relationship("Alice","Henry","working_for").
has_relationship("Bob","Cindy","secret_crush").
has_relationship("Bob","Henry","working_for").
has_relationship("Cindy","Alice","mutual_rivals").
has_relationship("Cindy","Georgia","sibling_of").
has_relationship("Cindy","John","mutual_rivals").
has_relationship("Fred","Henry","mentor_of").
has_relationship("Fred","Isabel","rivals").

In [87]:
%%writefile establishedFacts.lp

sceneLocation(messHall).
inscene("Alice").
inscene("Bob").

personality("Alice","primary","Boss").
personality("Alice","secondary","Pessimist").
personality("Bob","primary","Rugged").
personality("Bob","secondary","Resigned").

has_relationship("Alice","Cindy","mentor_of").
has_relationship("Bob","Cindy","working_for").

has_profession("Alice","giftshopkeep").
has_profession("Bob","skier").

has_modifier("Alice","in_hiding").
has_modifier("Bob","hates_this_place").

personality("Cindy","primary","Big Fun").
personality("Cindy","secondary","Jerk").
has_modifier("Cindy","is_dying").
has_relationship("Cindy","Isabel","professor_of").
has_profession("Cindy","professor").

has_opinion("Alice","Bob","dislike").
has_opinion("Bob","Alice","neutral").

player_input("greet","Bob").
detective_character("Alice").

Writing establishedFacts.lp


In [88]:
%%writefile storygeneration.lp
%This is the file for generating conversational moves that Agatha can take.
%Requires information from an establishedFacts.lp file to seed all the choices.

action("ignore",Dect,Char) :- player_input(_,Char), detective_character(Dect), has_opinion(Char,Dect,"dislike").
action("insult",Dect,Char) :- player_input(_,Char), detective_character(Dect), has_opinion(Char,Dect,"dislike").
    
{chosen_action(A,Dect,Char) : action(A,Dect,Char)} = 4.
#show chosen_action/3.

Writing storygeneration.lp
